In [5]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

mesh = ngs.Mesh('team10_2d.vol')
mesh.Curve(2)
# mesh.Refine()

print("using 3d mesh with ne=", mesh.ne, " elements and nv=", mesh.nv, " points")


using 3d mesh with ne= 45440  elements and nv= 23021  points


In [6]:
from ngsolve.krylovspace import CGSolver

deg = 0

# HCurl = ngs.HCurl(mesh, order = deg, dirichlet = "ambient_face")
HCurl = ngs.HCurl(mesh, order = deg)
H1 = ngs.H1(mesh, order = deg+1)

deg = 3

X = HCurl*H1

(u,p),(v,q) = X.TnT()

K = ngs.BilinearForm(X)
K += (ngs.curl(u)*ngs.curl(v) - v*ngs.grad(p) +ngs.grad(q)*u)*ngs.dx
C = ngs.Preconditioner(K, type = "local")

total_current = 1000/0.0025
cfJ = mesh.MaterialCF({"coil_plus": total_current, "coil_minus": -total_current}, default = 0)


f = ngs.LinearForm(X)
f += cfJ*ngs.curl(v)*ngs.dx()

with ngs.TaskManager():
    K.Assemble()
    f.Assemble()

HsA = ngs.GridFunction(X)
HsA.vec.data = K.mat.Inverse(X.FreeDofs())*f.vec

Hs = HsA.components[0]

# # with ngs.TaskManager():
# #     iterativeSolver = CGSolver(K.mat, pre = C.mat, tol  = 1e-8,  maxiter = 1000)
# #     J.vec.data = iterativeSolver * f.vec

# J.vec.FV()

NgException: PardisoInverse: Setup and Factorization failed.

In [ ]:
# Draw(Hs, mesh)
# Draw(ngs.curl(Hs), mesh)